In [120]:
import math
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import date, timedelta, datetime
import time
import talib as ta
from datetime import date
from dateutil.parser import parse
from pynse import *

In [121]:
nse=Nse()

In [122]:
def get_options_stocklist():
  df = pd.read_csv('options_list.csv')
  return df

In [123]:
# # download latest bhavcopy to get strike prices of all options

In [124]:
bhavcopy_fno = nse.bhavcopy_fno()
# drop unwanted columns
bhavcopy_fno = bhavcopy_fno.reset_index(level=0)
bhavcopy_fno = bhavcopy_fno[['SYMBOL','INSTRUMENT','EXPIRY_DT', 'STRIKE_PR', 'OPTION_TYP', 'CLOSE', 'OPEN_INT']]
# convert date format to desired format
#bhavcopy_fno['EXPIRY_DT'] =  bhavcopy_fno.apply(lambda x: parse(x.EXPIRY_DT), axis=1)

In [125]:
test=nse.option_chain('BERGEPAINT', expiry=date(2022, 4, 28)) # this is required due to the bug in library, has no other use

In [126]:
current_expiry = nse.expiry_list[1]

In [127]:
def manipulate_date(strdate):
    datetimeobj=datetime.strptime(strdate,"%d-%b-%Y")
    return datetimeobj.date()

In [128]:
def download_stock(symbol, start_date, end_date):
  start = start_date
  end = end_date
  symbol = symbol+'.NS'
  stock_data = yf.download(symbol, start=start, end=end,rounding=True)
  if(len(stock_data)<1):
    stock_data = pd.DataFrame()
  return stock_data

In [129]:
df_reverse = pd.read_csv('lunar_2022.csv')

In [130]:
def trend_reversal(): # this will give, if the trend changes tomorrow or not and also predict the trend reversal date
    current_date = date.today()
    previous_date = current_date - timedelta(days = 1)
    next_day = current_date + timedelta(days =1)

    # is the trend reversing tommorow?
    if(df_reverse['reversal_dates'].str.contains(str(current_date)).any()
        | df_reverse['reversal_dates'].str.contains(str(previous_date)).any() | df_reverse['reversal_dates'].str.contains(str(next_day)).any()):
        reverse_trend = 'yes'
    else:
        reverse_trend = 'no'
    
    #current trend will end on the following date..  
    # will give next reversal date, if the target is not achieved then square it off before this date
    # next trend reversal dates

    trend_end_date = datetime.strptime(df_reverse.loc[df_reverse['reversal_dates']>=str(current_date)].values[0][0], '%Y-%m-%d').date()
    return reverse_trend, trend_end_date   

In [131]:
# macd is fast ema - slow ema
# macdsignal is ema of macd
# hist = macd - macdsignal
def macd_strategy(data):
    data['macd'], data['macdsignal'], data['macdhist'] = ta.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    if(data['macd'][-1] > data['macdsignal'][-1]):
        signal ='buy'
    elif(data['macd'][-1] < data['macdsignal'][-1]):
        signal ='sell'
    else:
        signal ='none'
    
    return signal

In [132]:
# BB logic
# IF Prev_close > Prev_LOWERBB & Current_close < Current_LOWER_BB => buy
#IF Prev_close < Prev_UPPERBB & Current_close > Current_UPPER_BB => sell
def bb_strategy(data):
    Upperband, Middleband, Lowerband = ta.BBANDS(data['Close'], timeperiod=7, nbdevup=2, nbdevdn=2, matype=0)
    data['Upperband']= Upperband
    data['Lowerband'] = Lowerband
    data['Middleband'] = Middleband
    if((data['Close'][-2]>data['Lowerband'][-2]) and (data['Close'][-1] < data['Lowerband'][-1])):
        signal = 'buy'
    elif((data['Close'][-2]<data['Upperband'][-2]) and (data['Close'][-1] > data['Upperband'][-1])):
        signal = 'sell'
    else:
        signal = 'none'
    return signal

In [133]:
def rsi_strategy(data):
    RSI = ta.RSI(data['Close'], timeperiod=10)[-1]
    if(RSI < 30):
        signal = 'buy'
    elif(RSI > 70):
        signal ='sell'
    else:
        signal ='none'
    return signal

In [134]:
def decide_position(symbol):
    current_date = date.today()
    
    # Calculate EMA on current day
    data = download_stock(symbol,current_date - timedelta(days = 70),current_date+timedelta(days=1))

    # Get signals
    macd_signal = macd_strategy(data)
    bb_signal = bb_strategy(data)
    rsi_signal = rsi_strategy(data)

    trend_rev = trend_reversal() 

    if(macd_signal == 'buy' and rsi_signal == 'buy' and bb_signal == 'buy'):
        signal = 'buy' # meaning current trend is upward and the next trend is going to be downward
    elif(macd_signal == 'sell' and bb_signal=='sell'):
        signal = 'sell'  # meaning current trend is downward and the next trend is going to be upward
    else:
        signal = 'none'

    return signal, str(trend_rev[1]) # gives signal and the next trend reversal date

In [135]:
def Gann_prediction(LTP):
  
  # find sq root of the ltp
  sq_rt_ltp = math.sqrt(LTP)
  #ceil the sq rt
  cieled_sq_rt = math.ceil(sq_rt_ltp)
  #get starting number for iteration
  lowest_num = cieled_sq_rt-2
  block = [lowest_num, lowest_num+1, lowest_num+2]
  
  for i in (block):
    centre_1 = math.pow(i+0.125, 2)
    centre_2 = math.pow(i+0.125*2, 2)
    centre_3 = math.pow(i+0.125*3, 2)
    centre_4 = math.pow(i+0.125*4, 2)
    centre_5 = math.pow(i+0.125*5, 2)
    centre_6 = math.pow(i+0.125*6, 2)
    centre_7 = math.pow(i+0.125*7, 2)
    centre_8 = math.pow(i+0.125*8, 2)
    if(i==block[0]):
      level_1 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
    if(i==block[1]):
      level_2 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
    if(i==block[2]):
      level_3 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
  
  #merge list
  Gann_list = level_1 + level_2 + level_3


  insrt_loc = 0  
  for j in range(len(Gann_list)):
    if(Gann_list[j] < LTP < Gann_list[j+1]):
      insrt_loc = j+1
      Gann_list.insert(insrt_loc, LTP)
      break

  Resistance_1, Resistance_2, Resistance_3, Resistance_4 = (Gann_list[insrt_loc+2], Gann_list[insrt_loc+3],
                                                                       Gann_list[insrt_loc+4], Gann_list[insrt_loc+5])

  Support_1, Support_2, Support_3, Support_4 = (Gann_list[insrt_loc-2], Gann_list[insrt_loc-3],
                                                                       Gann_list[insrt_loc-4], Gann_list[insrt_loc-5])

  buy_above = Gann_list[insrt_loc+1]
  c1 = 0.99995
  sell_target_1, sell_target_2, sell_target_3, sell_target_4 =  (round(c1*Resistance_1,2), round(c1*Resistance_2,2), 
                                                                              round(1*Resistance_3,2), round(c1*Resistance_4,2))
  sl_long = Gann_list[insrt_loc-1]
  sell_below = Gann_list[insrt_loc-1]
  c2 = 1.00005
  buy_target_1, buy_target_2, buy_target_3, buy_target_4 = (round(c2*Support_1,2), round(c2*Support_2,2), round(c2*Support_3,2), 
                                                                        round(c2*Support_4,2))
  sl_short = Gann_list[insrt_loc+1]
  
  return (buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4,
          sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4)
  

In [136]:
def opt_Gann_prediction(LTP):
  
  # find sq root of the ltp
  sq_rt_ltp = math.sqrt(LTP)
  #ceil the sq rt
  cieled_sq_rt = math.ceil(sq_rt_ltp)
  #get starting number for iteration
  lowest_num = cieled_sq_rt-2
  block = [lowest_num, lowest_num+1, lowest_num+2]
  
  for i in (block):
    centre_1 = math.pow(i+0.125, 2)
    centre_2 = math.pow(i+0.125*2, 2)
    centre_3 = math.pow(i+0.125*3, 2)
    centre_4 = math.pow(i+0.125*4, 2)
    centre_5 = math.pow(i+0.125*5, 2)
    centre_6 = math.pow(i+0.125*6, 2)
    centre_7 = math.pow(i+0.125*7, 2)
    centre_8 = math.pow(i+0.125*8, 2)
    if(i==block[0]):
      level_1 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
    if(i==block[1]):
      level_2 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
    if(i==block[2]):
      level_3 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
  
  #merge list
  Gann_list = level_1 + level_2 + level_3


  insrt_loc = 0  
  for j in range(len(Gann_list)):
    if(Gann_list[j] < LTP < Gann_list[j+1]):
      insrt_loc = j+1
      Gann_list.insert(insrt_loc, LTP)
      break

  Resistance_1, Resistance_2, Resistance_3, Resistance_4, Resistance_5, Resistance_6 = (Gann_list[insrt_loc+2], Gann_list[insrt_loc+3],
                                                                       Gann_list[insrt_loc+4], Gann_list[insrt_loc+5], Gann_list[insrt_loc+6], Gann_list[insrt_loc+7])

  Support_1, Support_2, Support_3, Support_4, Support_5, Support_6 = (Gann_list[insrt_loc-2], Gann_list[insrt_loc-3],
                                                                       Gann_list[insrt_loc-4], Gann_list[insrt_loc-5],Gann_list[insrt_loc-6], Gann_list[insrt_loc-7])

  buy_above = Gann_list[insrt_loc+1]
  c1 = 0.99995
  sell_target_1, sell_target_2, sell_target_3, sell_target_4, sell_target_5, sell_target_6 =  (round(c1*Resistance_1,2), round(c1*Resistance_2,2), 
                                                                              round(c1*Resistance_3,2), round(c1*Resistance_4,2), round(c1*Resistance_5,2), round(c1*Resistance_6,2))
  sl_long = Gann_list[insrt_loc-1]
  sell_below = Gann_list[insrt_loc-1]
  c2 = 1.00005
  buy_target_1, buy_target_2, buy_target_3, buy_target_4, buy_target_5, buy_target_6 = (round(c2*Support_1,2), round(c2*Support_2,2), round(c2*Support_3,2), 
                                                                        round(c2*Support_4,2), round(c2*Support_5,2), round(c2*Support_6,2))
  sl_short = Gann_list[insrt_loc+1]
  
  return (buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4, sell_target_5,sell_target_6,
          sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4, buy_target_5, buy_target_6)
  

In [137]:
def profit_percentage(prediction_list_df): # this function will use multiple levls of filtering
    
    stock_tckr = prediction_list_df['stock_tckr']
    buy_price =  prediction_list_df['buy_above'] #buying price for long trade
    sell_price = prediction_list_df['sell_below'] # selling price for short trade
    sl_long = prediction_list_df['sl_long']
    sl_short = prediction_list_df['sl_short']

    # Percentage profit with target 1 achieved for long trade
    pct_profit_long_tgt_1 = (prediction_list_df['sell_target_1'] - buy_price) * 100/ buy_price
    # Percentage profit with target 2 achieved for long trade
    pct_profit_long_tgt_2 = (prediction_list_df['sell_target_2'] - buy_price) * 100/ buy_price
    # Percentage profit with target 3 achieved for long trade
    pct_profit_long_tgt_3 = (prediction_list_df['sell_target_3'] - buy_price) * 100/ buy_price
    # Percentage profit with target 4 achieved for long trade
    pct_profit_long_tgt_4 = (prediction_list_df['sell_target_4'] - buy_price) * 100/ buy_price

    # Percentage profit with target 1 achieved for short trade
    pct_profit_short_tgt_1 = (sell_price - prediction_list_df['buy_target_1']) *100/ prediction_list_df['buy_target_1']
    # Percentage profit with target 2 achieved for short trade
    pct_profit_short_tgt_2 = (sell_price - prediction_list_df['buy_target_2']) *100/ prediction_list_df['buy_target_2']
    # Percentage profit with target 3 achieved for short trade
    pct_profit_short_tgt_3 = (sell_price - prediction_list_df['buy_target_3']) *100/ prediction_list_df['buy_target_3']
    # Percentage profit with target 4 achieved for short trade
    pct_profit_short_tgt_4 = (sell_price - prediction_list_df['buy_target_4']) *100/ prediction_list_df['buy_target_4']
    
    # check stop loss vs pct profit
    # Risk reward percentage: long trade vs stop loss
    loss_pct_against_long_trade = (buy_price - sl_long) * 100/ buy_price
    # Risk reward percentage: short trade vs stop loss
    loss_pct_against_short_trade = (sl_short - sell_price) * 100/ sl_short

    return (stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,
            pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,
            loss_pct_against_long_trade, loss_pct_against_short_trade)

In [138]:
def opt_profit_percentage(prediction_list_df): # this function will use multiple levls of filtering
    
    stock_tckr = prediction_list_df['stock_tckr']
    buy_price =  prediction_list_df['buy_above'] #buying price for long trade
    sell_price = prediction_list_df['sell_below'] # selling price for short trade
    sl_long = prediction_list_df['sl_long']
    sl_short = prediction_list_df['sl_short']

    # Percentage profit with target 1 achieved for long trade
    pct_profit_long_tgt_1 = (prediction_list_df['sell_target_1'] - buy_price) * 100/ buy_price
    # Percentage profit with target 2 achieved for long trade
    pct_profit_long_tgt_2 = (prediction_list_df['sell_target_2'] - buy_price) * 100/ buy_price
    # Percentage profit with target 3 achieved for long trade
    pct_profit_long_tgt_3 = (prediction_list_df['sell_target_3'] - buy_price) * 100/ buy_price
    # Percentage profit with target 4 achieved for long trade
    pct_profit_long_tgt_4 = (prediction_list_df['sell_target_4'] - buy_price) * 100/ buy_price
    # Percentage profit with target 3 achieved for long trade
    pct_profit_long_tgt_5 = (prediction_list_df['sell_target_5'] - buy_price) * 100/ buy_price
    # Percentage profit with target 4 achieved for long trade
    pct_profit_long_tgt_6 = (prediction_list_df['sell_target_6'] - buy_price) * 100/ buy_price

    # Percentage profit with target 1 achieved for short trade
    pct_profit_short_tgt_1 = (sell_price - prediction_list_df['buy_target_1']) *100/ prediction_list_df['buy_target_1']
    # Percentage profit with target 2 achieved for short trade
    pct_profit_short_tgt_2 = (sell_price - prediction_list_df['buy_target_2']) *100/ prediction_list_df['buy_target_2']
    # Percentage profit with target 3 achieved for short trade
    pct_profit_short_tgt_3 = (sell_price - prediction_list_df['buy_target_3']) *100/ prediction_list_df['buy_target_3']
    # Percentage profit with target 4 achieved for short trade
    pct_profit_short_tgt_4 = (sell_price - prediction_list_df['buy_target_4']) *100/ prediction_list_df['buy_target_4']
      # Percentage profit with target 3 achieved for short trade
    pct_profit_short_tgt_5 = (sell_price - prediction_list_df['buy_target_5']) *100/ prediction_list_df['buy_target_5']
    # Percentage profit with target 4 achieved for short trade
    pct_profit_short_tgt_6 = (sell_price - prediction_list_df['buy_target_6']) *100/ prediction_list_df['buy_target_6']
    
    # check stop loss vs pct profit
    # Risk reward percentage: long trade vs stop loss
    loss_pct_against_long_trade = (buy_price - sl_long) * 100/ buy_price
    # Risk reward percentage: short trade vs stop loss
    loss_pct_against_short_trade = (sl_short - sell_price) * 100/ sl_short

    return (stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,pct_profit_long_tgt_5, pct_profit_long_tgt_6,
            pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,pct_profit_short_tgt_5, pct_profit_short_tgt_6,
            loss_pct_against_long_trade, loss_pct_against_short_trade)

In [139]:
current_expiry

datetime.date(2022, 5, 26)

In [140]:
# For buy signals from the signal list add target 1 profit % to ltp and get strike price. Strike price won't be exact so find a nearest strike price
def get_strike_price(stock_tckr):
    #expiry=current_expiry.strftime('%Y-%m-%d')
    try:
        CE_strike_price=bhavcopy_fno.loc[bhavcopy_fno.query('SYMBOL == @stock_tckr & EXPIRY_DT == @current_expiry & OPTION_TYP =="CE"')['OPEN_INT'].idxmax()]['STRIKE_PR']
        PE_strike_price=bhavcopy_fno.loc[bhavcopy_fno.query('SYMBOL == @stock_tckr & EXPIRY_DT == @current_expiry & OPTION_TYP =="PE"')['OPEN_INT'].idxmax()]['STRIKE_PR']
    except:
        CE_strike_price, PE_strike_price = 0, 0
    return CE_strike_price, PE_strike_price

In [141]:
def stock_filtering(long_profit, long_loss, short_profit, short_loss):
    if((long_profit>(2*long_loss) or short_profit>(2*short_loss)) and (long_profit>2 or short_profit>2)):
        return "good_stock"
    else:
        return "remove_stock"


In [142]:
# Main 
options_stock_list = get_options_stocklist()
#risk_rewards = []
current_date = date.today() + timedelta(days=1) # this adjustment is required because python does not include last date of the range in yfinance
start_date = date.today() - timedelta(days=1) # previous 1 day from today


Get direction of stocks

In [143]:
# Swing trades
signal_list =[]
for i in range(len(options_stock_list)):
  #for i in range(0,10):
  options_stock_tckr = options_stock_list['symbol'][i] # get ticker symbol

  # call decide postion and trend reversal function
  signal, trend_reversal_date = decide_position(options_stock_tckr)
  signal_list.append([options_stock_tckr, signal, trend_reversal_date])

#create df of signal direction
signal_list_df = pd.DataFrame(signal_list, columns =['stock_tckr', 'signal', 'trend_reversal_date'])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [144]:
# Filter stocks with no signals and get a fresh list
stock_options_filtered = []
for i in range(len(signal_list_df)):
    if(signal_list_df['signal'][i]!='none'):
        stock_options_filtered.append([signal_list_df['stock_tckr'][i], signal_list_df['signal'][i]])
    else:
        pass

stock_options_filtered_df = pd.DataFrame(stock_options_filtered, columns =['stock_tckr', 'signal'])



In [145]:
# do a gann prediciion for equity price and get profit percentages only
# Gann prediction algorithm
prediction_list=[]
for i in range(len(stock_options_filtered_df)):
#for i in range(0,10): # test with 3 stocks
  option_stock_tckr = stock_options_filtered_df['stock_tckr'][i] # get ticker symbol
  option_stock_data = download_stock(option_stock_tckr, start_date ,current_date) # download data from yfinance 
  if(len(option_stock_data)>0 and not(pd.isnull(option_stock_data.iloc[0,3]))):
    current_day_close = round(option_stock_data.values[-1][3],2)

    (buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4,
           sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4)= Gann_prediction(current_day_close)
          
    prediction_list.append([option_stock_tckr, current_day_close, buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4,
           sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4])

prediction_list_df = pd.DataFrame(prediction_list, columns=['stock_tckr','current_day_close','buy_above', 'sl_long', 'sell_target_1', 'sell_target_2', 
        'sell_target_3', 'sell_target_4', 'sell_below', 'sl_short', 'buy_target_1', 'buy_target_2', 'buy_target_3', 'buy_target_4'])



In [146]:
#check profit percentage for target 2 of both buy and sell categories
#filtering low profit stocks
risk_rewards=[]
#for i in range(0,100): # test with 3 stocks
for i in range(len(prediction_list_df)):
  (stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,
  pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,
  loss_pct_against_long_trade, loss_pct_against_short_trade) = profit_percentage(prediction_list_df.iloc[i])

  risk_rewards.append([stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,
  pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,
  loss_pct_against_long_trade, loss_pct_against_short_trade])

risk_rewards_df = pd.DataFrame(risk_rewards, columns = ['stock_tckr', 'pct_profit_long_tgt_1', 'pct_profit_long_tgt_2', 'pct_profit_long_tgt_3',
 'pct_profit_long_tgt_4', 'pct_profit_short_tgt_1', 'pct_profit_short_tgt_2', 'pct_profit_short_tgt_3', 'pct_profit_short_tgt_4', 
 'loss_pct_against_long_trade', 'loss_pct_against_short_trade'])

# Merge both data frames, Gann predictions and risk rewards for evaluation
reduced_stocks_df = pd.merge(prediction_list_df, risk_rewards_df,  on = 'stock_tckr')


In [147]:
# stock_options_filtered_df merge with reduced_stocks_df
reduced_stocks_df = pd.merge(stock_options_filtered_df,reduced_stocks_df, on = 'stock_tckr')

In [148]:
reduced_stocks_df

,signal,current_day_close,buy_above,sl_long,sell_target_1,sell_target_2,sell_target_3,sell_target_4,sell_below,sl_short,...,pct_profit_long_tgt_1,pct_profit_long_tgt_2,pct_profit_long_tgt_3,pct_profit_long_tgt_4,pct_profit_short_tgt_1,pct_profit_short_tgt_2,pct_profit_short_tgt_3,pct_profit_short_tgt_4,loss_pct_against_long_trade,loss_pct_against_short_trade


In [149]:
# Use Gann to predict using ltp option price
# do a gann prediciion for option price and get profit percentages only
# Gann prediction algorithm
opt_prediction_list=[]
for i in range(len(reduced_stocks_df)):
#for i in range(0,10): # test with 3 stocks
  option_stock_tckr = reduced_stocks_df['stock_tckr'][i] # get ticker symbol
  signal = reduced_stocks_df['signal'][i]
  print(reduced_stocks_df['stock_tckr'][i])
  CE_strike_price, PE_strike_price = get_strike_price(reduced_stocks_df['stock_tckr'][i])

# based on strike price, get last traded price(option price this time)
  if(CE_strike_price!=0 and PE_strike_price!=0): # cos if the symbol return empty values, this check ignores the error
    if(signal == 'buy'):
      ltp_options=bhavcopy_fno.query("SYMBOL == @option_stock_tckr & EXPIRY_DT==@current_expiry & STRIKE_PR == @CE_strike_price & OPTION_TYP=='CE'").values[0][5]
    if(signal == 'sell'):
      ltp_options=bhavcopy_fno.query("SYMBOL == @option_stock_tckr & EXPIRY_DT==@current_expiry & STRIKE_PR == @PE_strike_price & OPTION_TYP=='PE'").values[0][5]

  # now use gann to make predictions on opt prices
    (buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4,sell_target_5,sell_target_6,
           sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4, buy_target_5, buy_target_6)= opt_Gann_prediction(ltp_options)
          
    opt_prediction_list.append([option_stock_tckr,CE_strike_price, PE_strike_price, current_expiry, ltp_options, buy_above, sl_long*0.5, sell_target_1, sell_target_2, sell_target_3, sell_target_4,sell_target_5, sell_target_6,
           sell_below, sl_short*0.5, buy_target_1, buy_target_2, buy_target_3, buy_target_4,buy_target_5, buy_target_6, signal])

options_prediction_list_df = pd.DataFrame(opt_prediction_list, columns=['stock_tckr','CE strike price', 'PE strike price', 'Expiry', 'Last traded price','buy_above', 'sl_long', 'sell_target_1', 'sell_target_2', 
        'sell_target_3', 'sell_target_4','sell_target_5', 'sell_target_6', 'sell_below', 'sl_short', 'buy_target_1', 'buy_target_2', 'buy_target_3', 'buy_target_4', 'buy_target_5', 'buy_target_6','Signal'])




In [150]:
# get profit percentage
#check profit percentage for target 2 of both buy and sell categories
#filtering low profit stocks
opt_risk_rewards=[]
#for i in range(0,100): # test with 3 stocks
for i in range(len(options_prediction_list_df)):
  (stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,pct_profit_long_tgt_5, pct_profit_long_tgt_6,
  pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4, pct_profit_short_tgt_5, pct_profit_short_tgt_6,
  loss_pct_against_long_trade, loss_pct_against_short_trade) = opt_profit_percentage(options_prediction_list_df.iloc[i])

  opt_risk_rewards.append([stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4, pct_profit_long_tgt_5, pct_profit_long_tgt_6,
  pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,pct_profit_short_tgt_5, pct_profit_short_tgt_6,
  loss_pct_against_long_trade, loss_pct_against_short_trade])

opt_risk_rewards_df = pd.DataFrame(opt_risk_rewards, columns = ['stock_tckr', 'pct_profit_long_tgt_1', 'pct_profit_long_tgt_2', 'pct_profit_long_tgt_3',
 'pct_profit_long_tgt_4','pct_profit_long_tgt_5','pct_profit_long_tgt_6', 'pct_profit_short_tgt_1', 'pct_profit_short_tgt_2', 'pct_profit_short_tgt_3', 'pct_profit_short_tgt_4', 'pct_profit_short_tgt_5', 'pct_profit_short_tgt_6', 
 'loss_pct_against_long_trade', 'loss_pct_against_short_trade'])

# Merge both data frames, Gann predictions and risk rewards for evaluation
opt_reduced_stocks_df = pd.merge(options_prediction_list_df, opt_risk_rewards_df,  on = 'stock_tckr')


In [151]:
# Now as we do not plan to short either CE or PE. If the stock(EQ) says 'buy' then you buy CE. If stock signal(EQ) says 'sell' then you buy put- thats how it is designed and coded. The reason it gives all other details is because the functions are coded to be reused to all models.
# So we drop the short signals from the dataframe above as they are redundant. Alternative fresh code can be written to optimise but at this point it is not required.

opt_reduced_stocks_df.drop(['sell_below',
       'sl_short', 'buy_target_1', 'buy_target_2', 'buy_target_3',
       'buy_target_4', 'buy_target_5',
       'buy_target_6','pct_profit_short_tgt_1',
       'pct_profit_short_tgt_2', 'pct_profit_short_tgt_3',
       'pct_profit_short_tgt_4', 'pct_profit_short_tgt_5','pct_profit_short_tgt_6','loss_pct_against_short_trade'], axis=1, inplace = True)


In [152]:
# Drop all profit % and keep the max gain only
opt_reduced_stocks_df.drop(['pct_profit_long_tgt_1', 'pct_profit_long_tgt_2', 'pct_profit_long_tgt_3','pct_profit_long_tgt_4', 'pct_profit_long_tgt_5', 'loss_pct_against_long_trade'], axis=1, inplace=True)

In [153]:
opt_reduced_stocks_df.rename(columns={'stock_tckr': 'Stock symbol', 'buy_above': 'Buy above', 
'sl_long': 'Stop loss', 'sell_target_1':'Sell target 1','sell_target_2':'Sell target 2','sell_target_3':'Sell target 3',
'sell_target_4':'Sell target 4', 'sell_target_5':'Sell target 5','sell_target_6':'Sell target 6','pct_profit_long_tgt_6':'Max Gain(%)' }, inplace=True)

In [154]:
tomorrow=date.today() + timedelta(days=1)

In [155]:
opt_reduced_stocks_df.insert(loc = 0,
          column = 'Date',
          value = tomorrow)

In [156]:
opt_reduced_stocks_df

,Date,CE strike price,PE strike price,Expiry,Last traded price,Buy above,Stop loss,Sell target 1,Sell target 2,Sell target 3,Sell target 4,Sell target 5,Sell target 6,Signal,Stock symbol,Max Gain(%)


In [157]:
opt_reduced_stocks_df.to_csv('Predictions/opt_predictions_tomorrow.csv')

In [158]:
# def decide_position(symbol):
#     current_date = date.today()

#     # previous moon date finder
#     previous_trend_start_date = df_reverse.loc[df_reverse['reversal_dates']<=str(current_date)].values[-1][0]
#     previous_trend_start_date=datetime.strptime(previous_trend_start_date, '%Y-%m-%d').date()
    
#     # get closing price of that day
    
#     stock_data_prev_tren_start_day = download_stock(symbol,previous_trend_start_date,previous_trend_start_date + timedelta(days=1))
#     if(len(stock_data_prev_tren_start_day)<1):
#         stock_data_prev_tren_start_day = download_stock(symbol,previous_trend_start_date,previous_trend_start_date + timedelta(days=3))
   
#     # Get EMA on previous moon day
#     data_prev_moon = download_stock(symbol,previous_trend_start_date - timedelta(days = 20),previous_trend_start_date + timedelta(days=1))
   
#     if(len(data_prev_moon)>0):
#     # calculate EMA and decide the direction
#         ema_prev_moon = ta.EMA(data_prev_moon.Close, timeperiod=7)
    

#     # Calculate EMA on current day
#     data = download_stock(symbol,current_date - timedelta(days = 20),current_date+timedelta(days=1))
#     ltp = data['Close'][-1]
    
#     if(len(data)>0):
#         ema_current = ta.EMA(data.Close, timeperiod=7)

#     trend_rev = trend_reversal() 

#     # check if the current trend is an upward trend or downward trend
#     #if(current close  > previous moon close)and ema on prev moon(+ or - 1) < ema current date then short it, if trend reversal is yes
#     if((ltp > stock_data_prev_tren_start_day['Close'][-1]) & (ema_prev_moon[-1] < ema_current[-1])):
#         signal = 'buy' # meaning current trend is upward and the next trend is going to be downward
#     if((ltp < stock_data_prev_tren_start_day['Close'][-1]) & (ema_prev_moon[-1] > ema_current[-1])):
#         signal = 'sell'  # meaning current trend is downward and the next trend is going to be upward
#     else:
#         signal = 'none'

#     return signal, str(trend_rev[1]) # gives signal and the next trend reversal date